Google Colab link : https://colab.research.google.com/drive/120IqNdb_44K1kSgqTTsZosf7VOaLO0ki?usp=sharing


# Preliminaries

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

Dataset link : https://www.kaggle.com/datasets/tunguz/big-five-personality-test

In [3]:
dataset = pd.read_csv("/content/drive/MyDrive/SEM4/Research Method/RM Kel 19 Experiment/data-final.csv", sep='\t')

print(dataset.shape)
dataset.head()

(1015341, 110)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11,1,MY,3.1698,101.706
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7,1,GB,51.75,-1.25
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,2016-03-03 02:02:57,1366.0,768.0,8.0,315.0,17,2,KE,1.0,38.0


# Data Preprocessing

In [4]:
data = dataset.drop(list(dataset)[50:], axis=1)

print(data.shape)
data.head()

(1015341, 50)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,5.0,1.0,5.0,1.0,5.0,1.0,5.0,3.0,5.0,5.0


In [5]:
for i in data.columns:
  data = data[(data[i].notna()) & (data[i] != 0)]

print(data.shape)

(874434, 50)


In [6]:
x = data.drop(columns=['EST9'])
y = data['EST9']

### Random oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
data['EST9'].value_counts()

4.0    247851
2.0    199050
3.0    182001
5.0    133152
1.0    112380
Name: EST9, dtype: int64

In [ ]:
oversampler = RandomOverSampler(random_state=42)

In [ ]:
x_resampled, y_resampled = oversampler.fit_resample(x, y)

In [ ]:
y_resampled.value_counts()

3.0    247851
1.0    247851
4.0    247851
2.0    247851
5.0    247851
Name: EST9, dtype: int64

### Shaping dataset

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [8]:
timesteps = 1
input_dim = 49

x_1 = np.resize(x, (x.shape[0], timesteps, input_dim))
x_train_1 = np.resize(x_train, (x_train.shape[0], timesteps, input_dim))
x_test_1 = np.resize(x_test, (x_test.shape[0], timesteps, input_dim))

In [9]:
y_train_1 = y_train - 1
y_test_1 = y_test - 1

In [10]:
y_train_1 = to_categorical(y_train_1)
y_test_1 = to_categorical(y_test_1)

print(y_test_1)

[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]]


# LSTM training

In [11]:
x_train_1.shape

(786990, 1, 49)

In [12]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(x_train_1, y_train_1, epochs=10, batch_size=256, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=7, min_delta=0.0001)])

Epoch 1/10
2460/2460 [==============================] - 26s 7ms/step - loss: 1.2533 - accuracy: 0.4572 - val_loss: 1.1953 - val_accuracy: 0.4851
Epoch 2/10
2460/2460 [==============================] - 16s 6ms/step - loss: 1.2170 - accuracy: 0.4784 - val_loss: 1.1815 - val_accuracy: 0.4940
Epoch 3/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.2044 - accuracy: 0.4854 - val_loss: 1.1698 - val_accuracy: 0.5016
Epoch 4/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1980 - accuracy: 0.4900 - val_loss: 1.1681 - val_accuracy: 0.5023
Epoch 5/10
2460/2460 [==============================] - 16s 6ms/step - loss: 1.1934 - accuracy: 0.4930 - val_loss: 1.1590 - val_accuracy: 0.5103
Epoch 6/10
2460/2460 [==============================] - 16s 7ms/step - loss: 1.1908 - accuracy: 0.4941 - val_loss: 1.1581 - val_accuracy: 0.5084
Epoch 7/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1882 - accuracy: 0.4963 - val_loss: 1.1592 - val_ac

In [15]:
model.evaluate(x_test_1,y_test_1)

2733/2733 [==============================] - 9s 3ms/step - loss: 1.1517 - accuracy: 0.5155


[1.1517356634140015, 0.5154727697372437]

# GRU training

In [ ]:
modelGRU = Sequential()
modelGRU.add(GRU(128, return_sequences=True, input_shape=(timesteps, input_dim)))
modelGRU.add(Dropout(0.5))
modelGRU.add(GRU(64))
modelGRU.add(Dropout(0.5))
modelGRU.add(Dense(5, activation='softmax'))

In [ ]:
modelGRU.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
modelGRU.fit(x_train_1, y_train_1, epochs=10, batch_size=256, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=7, min_delta=0.0001)])

Epoch 1/10
2187/2187 [==============================] - 35s 14ms/step - loss: 1.2584 - accuracy: 0.4530 - val_loss: 1.1986 - val_accuracy: 0.4873
Epoch 2/10
2187/2187 [==============================] - 30s 14ms/step - loss: 1.2202 - accuracy: 0.4769 - val_loss: 1.1877 - val_accuracy: 0.4910
Epoch 3/10
2187/2187 [==============================] - 29s 13ms/step - loss: 1.2064 - accuracy: 0.4845 - val_loss: 1.1845 - val_accuracy: 0.4915
Epoch 4/10
2187/2187 [==============================] - 29s 13ms/step - loss: 1.1985 - accuracy: 0.4899 - val_loss: 1.1648 - val_accuracy: 0.5061
Epoch 5/10
2187/2187 [==============================] - 29s 13ms/step - loss: 1.1939 - accuracy: 0.4928 - val_loss: 1.1585 - val_accuracy: 0.5094
Epoch 6/10
2187/2187 [==============================] - 29s 13ms/step - loss: 1.1896 - accuracy: 0.4959 - val_loss: 1.1610 - val_accuracy: 0.5093
Epoch 7/10
2187/2187 [==============================] - 29s 13ms/step - loss: 1.1871 - accuracy: 0.4968 - val_loss: 1.1610 -

In [ ]:
modelGRU.evaluate(x_test_1,y_test_1)

5466/5466 [==============================] - 11s 2ms/step - loss: 1.1514 - accuracy: 0.5137


[1.1514365673065186, 0.5137374401092529]